In [11]:
import os
import glob
import time
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from doa_py import estimation, simulation_room

In [15]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)
# Abrir el GSpreadsheet
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=1166684204#gid=1166684204")

In [16]:
data_input = 'input_sim_1' #nombre de la hoja de datos de entrada

sheet = spreadsheet.worksheet(data_input) # Abrir la hoja de cálculo con los datos de entrada
data = sheet.get_all_records()

#Ahora puedo manejar todos los datos con el DataFrame de pandas sin modificar la hoja de cálculo
df_simulations = pd.DataFrame(data)
df_simulations.head() # Verifico lo importado

,description,simulation_name,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,src_ang_start,src_ang_end,src_ang_step
0,Variación de la distancia de la fuente al cent...,dist_source_to_array_1m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,1,1.2,0,180,10
1,Variación de la distancia de la fuente al cent...,dist_source_to_array_2m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,2,1.2,0,180,10
2,Variación de la distancia de la fuente al cent...,dist_source_to_array_3m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,3,1.2,0,180,10
3,Variación de la distancia de la fuente al cent...,dist_source_to_array_4m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,4,1.2,0,180,10
4,Variación de la distancia de la fuente al cent...,dist_source_to_array_5m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,5,1.2,0,180,10


In [17]:
df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla
df_simulations.head() # Verificar el resultado

,description,simulation_name,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,src_ang_start,src_ang_end,src_ang_step,room_dim
0,Variación de la distancia de la fuente al cent...,dist_source_to_array_1m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,1,1.2,0,180,10,"(35, 45, 18)"
1,Variación de la distancia de la fuente al cent...,dist_source_to_array_2m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,2,1.2,0,180,10,"(35, 45, 18)"
2,Variación de la distancia de la fuente al cent...,dist_source_to_array_3m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,3,1.2,0,180,10,"(35, 45, 18)"
3,Variación de la distancia de la fuente al cent...,dist_source_to_array_4m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,4,1.2,0,180,10,"(35, 45, 18)"
4,Variación de la distancia de la fuente al cent...,dist_source_to_array_5m_voice,audio_anecoico_corto.wav,35,45,18,2,90,4,0.1,1.2,omni,5,1.2,0,180,10,"(35, 45, 18)"


## Simulación de audios captados por micrófonos 

In [18]:
for idx, sim in df_simulations.iterrows():
    # solo una vez por simulación
    room, center = simulation_room.build_room_Nmics(
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"]
    )

    for angle in range(sim['src_ang_start'], sim['src_ang_end'] + 1, sim['src_ang_step']):
        src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
            wav_path=f'audios/anechoic/{sim["audio"]}',
            out_dir=f'audios/output/{sim["simulation_name"]}/{angle}',
            audio_name=f'{angle}_{sim["simulation_name"]}',
            fs=48000,
            room_dim=sim["room_dim"],
            rt60=sim["rt60"],
            snr_db=sim["snr_db"],
            n_mics=sim["n_mics"],
            mic_d=sim["mic_d"],
            mic_z=sim["mic_z"],
            mic_directivity=sim["mic_directivity"],
            src_dist=sim["src_dist"],
            src_az_deg=angle,
            src_z=sim["src_z"],
            save_audio=True,
            prebuilt_room=room,
            room_center=center
        )


## Calcular DOA desde archivos WAV

In [19]:
methods = ['classic', 'scot', 'roth', 'phat']  # Método de estimación a utilizar

for method in methods:
    for idx, sim in df_simulations.iterrows(): # Itero cada simulación
        # Cargar los archivos de audio generados
        simulations = os.listdir(f'audios/output/{sim["simulation_name"]}')
        # Creación del DataFrame para almacenar los resultados
        df_results = pd.DataFrame(np.repeat(df_simulations.iloc[idx:idx+1, 2:14].values, len(simulations), axis=0), columns=df_simulations.columns[2:14])

        row = 0  
        for angle in simulations:
            # Obtener las señales de cada posición simulada
            mic_signals = glob.glob(f'audios/output/{sim["simulation_name"]}/{angle}/*.wav')
            avg_angle, avg_tdoa, angles_per_mic_ref, tdoas_per_mic_ref = estimation.estimate_doa(
            signals_input=mic_signals,
            d=sim["mic_d"],
            fs=48000,
            method=method,
            verbose=False
            )
            
            df_results.at[row, 'method'] = method
            df_results.at[row, 'angle'] = int(angle)
            df_results.at[row, 'avg_angle'] = avg_angle
            df_results.at[row, 'angle_error'] = np.abs(avg_angle - int(angle))
            df_results.at[row, 'desv_std_angle'] = np.std([angle for angles in angles_per_mic_ref for angle in angles])
            
            row += 1
            
        # Ordenar valores de df
        sheet_name = f'{sim["simulation_name"]}_{method}'
        # df_results['angle'] = pd.to_numeric(df_results['angle'], errors='coerce')
        df_results = df_results.sort_values(by=['angle']).reset_index(drop=True)

        # Crear csv en una carpeta llamada csv_results
        if not os.path.exists('csv_results'):
            os.makedirs('csv_results')
            
        df_results.to_csv(f'csv_results/{sheet_name}.csv')
        # Subir a GSpreadsheet creando una nueva hoja
        
        try:
            spreadsheet.add_worksheet(title=f'{sheet_name}', rows="100", cols="20")
            results_sheet = spreadsheet.worksheet(f'{sheet_name}')
            results_sheet.update([df_results.columns.values.tolist()] + df_results.values.tolist())
        except Exception as e:
            print(f"Error al subir los resultados a Google Sheets: {e}, ver si no existe la hoja ya en el GSpreadsheet")    


In [6]:
methods = ['classic', 'scot', 'roth', 'phat']

for method in methods:
    print(f"\n⏳ Ejecutando método: {method}...")

    for idx, sim in df_simulations.iterrows():
        simulations = os.listdir(f'audios/output/{sim["simulation_name"]}')
        df_results = pd.DataFrame(
            np.repeat(df_simulations.iloc[idx:idx+1, 2:14].values, len(simulations), axis=0),
            columns=df_simulations.columns[2:14]
        )
        df_results['method'] = method
        df_results['angle'] = np.nan  # se completará más adelante
        df_results['avg_angle'] = np.nan
        df_results['angle_error'] = np.nan
        df_results['desv_std_angle'] = np.nan
        df_results['tiempo_angulo_s'] = np.nan

        row = 0
        for angle in simulations:
            mic_signals = glob.glob(f'audios/output/{sim["simulation_name"]}/{angle}/*.wav')
            if len(mic_signals) < 2:
                print(f"⚠️  Skipping angle {angle}: menos de 2 micrófonos")
                continue

            t0 = time.perf_counter()
            avg_angle, avg_tdoa, angles_per_mic_ref, tdoas_per_mic_ref = estimation.estimate_doa(
                signals_input=mic_signals,
                d=sim["mic_d"],
                fs=48000,
                method=method,
                n_jobs=8,
                verbose=False
            )
            elapsed = time.perf_counter() - t0

            df_results.at[row, 'angle'] = int(angle)
            df_results.at[row, 'avg_angle'] = avg_angle
            df_results.at[row, 'angle_error'] = np.abs(avg_angle - int(angle))
            df_results.at[row, 'desv_std_angle'] = np.std([a for angles in angles_per_mic_ref for a in angles])
            df_results.at[row, 'tiempo_angulo_s'] = round(elapsed, 4)
            row += 1

        df_results = df_results.dropna(subset=['angle']).copy()
        df_results = df_results.sort_values(by='angle').reset_index(drop=True)

        sheet_name = f'{sim["simulation_name"]}_{method}'
        if not os.path.exists('csv_results'):
            os.makedirs('csv_results')

        df_results.to_csv(f'csv_results/{sheet_name}.csv', index=False)

        try:
            spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")
            results_sheet = spreadsheet.worksheet(sheet_name)
            results_sheet.update([df_results.columns.values.tolist()] + df_results.astype(str).values.tolist())
        except Exception as e:
            print(f"Error al subir los resultados a Google Sheets: {e}")



⏳ Ejecutando método: classic...
Error al subir los resultados a Google Sheets: APIError: [400]: Invalid requests[0].addSheet: A sheet with the name "sweep_0_180_5_05_rev_classic" already exists. Please enter another name.
Error al subir los resultados a Google Sheets: APIError: [400]: Invalid requests[0].addSheet: A sheet with the name "sweep_0_180_5_1_rev_classic" already exists. Please enter another name.
Error al subir los resultados a Google Sheets: APIError: [400]: Invalid requests[0].addSheet: A sheet with the name "sweep_0_180_5_15_rev_classic" already exists. Please enter another name.
Error al subir los resultados a Google Sheets: APIError: [400]: Invalid requests[0].addSheet: A sheet with the name "sweep_0_180_5_2_rev_classic" already exists. Please enter another name.


: 